In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")
root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from data_sources.clob import CLOBDataSource

clob = CLOBDataSource()

clob.load_candles_cache(root_path + "/data")
candles = {f"{key[0]}_{key[1]}_{key[2]}": value.data for key, value in clob.candles_cache.items()}
for key, df in candles.items():
    df.reset_index(inplace=True, drop=True)
    bb_columns = [col for col in df.columns if "bb" in col.lower() or col in ["signal", "out_of_bounds", "score"]]
    df.drop(columns=bb_columns, inplace=True)

In [ ]:
from hummingbot.strategy_v2.backtesting.controllers_backtesting.directional_trading_backtesting import \
    DirectionalTradingBacktesting
import os
import datetime

start_time = datetime.datetime(2024, 8, 1, 21, 0, 0).timestamp()
end_time = datetime.datetime(2024, 8, 20, 18, 0, 0).timestamp()

controllers_conf_dir_path = os.path.join(root_path, "research_notebooks", "xtreet_bb", "configs")
config_files = os.listdir(controllers_conf_dir_path)
backtesting_engine = DirectionalTradingBacktesting()
backtesting_engine.backtesting_data_provider.candles_feeds = candles
backtesting_engine.backtesting_data_provider.start_time = int(start_time)
backtesting_engine.backtesting_data_provider.end_time = int(end_time)


In [ ]:
# bt_results = {}
# config_file = "shupoalgo.yml"
# config = backtesting_engine.get_controller_config_instance_from_yml(
#     config_path=config_file,
#     controllers_conf_dir_path=controllers_conf_dir_path,
#     controllers_module="controllers")
# backtesting_results = await backtesting_engine.run_backtesting(
#     controller_config=config, trade_cost=0.0006,
#     start=int(start_time), end=int(end_time), backtesting_resolution="1m")
# df = backtesting_results["processed_data"]["features"]
# executors = backtesting_results["executors"]
# results = backtesting_results["results"]
# print(f"Net PNL quote: {results['net_pnl_quote']} | Total Volume: {results['total_volume']} ")
# bt_results[config_file] = {"df": df, "executors": executors, "results": results, "config": config}

In [ ]:
# bt_results[config_file]["df"]

In [ ]:
bt_results = {}

for i, config_file in enumerate(config_files):
    print(f"Experiment {i}/{len(config_files)}: {config_file}")
    try:
        config = backtesting_engine.get_controller_config_instance_from_yml(
            config_path=config_file,
            controllers_conf_dir_path=controllers_conf_dir_path,
            controllers_module="controllers")
        backtesting_results = await backtesting_engine.run_backtesting(
            controller_config=config, trade_cost=0.0006,
            start=int(start_time), end=int(end_time), backtesting_resolution="1m")
        df = backtesting_results["processed_data"]["features"]
        executors = backtesting_results["executors"]
        results = backtesting_results["results"]
        print(f"Net PNL quote: {results['net_pnl_quote']} | Total Volume: {results['total_volume']} ")
        bt_results[config_file] = {"df": df, "executors": executors, "results": results, "config": config}
    except:
        print(f"Error with {config_file}")

In [ ]:
import pickle
for result in bt_results.values():
    result["results"]["trading_pair"] = result["config"].trading_pair
    result["results"]["config"] = result["config"].dict()
path = os.path.join(root_path, "research_notebooks", "xtreet_bb", "backtesting_results_2.pickle")
with open("backtesting_results_ts_nuevo.pickle", "wb") as f:
    pickle.dump(bt_results, f)

In [ ]:
import pandas as pd

custom_results = bt_results.copy()
for result in custom_results.values():
    result["results"]["trading_pair"] = result["config"].trading_pair
    result["results"]["config"] = result["config"].dict()
cols_to_show = ["net_pnl_quote", "total_volume", "close_types", "sharpe_ratio", "profit_factor",
                "max_drawdown_usd"]
results_df = pd.DataFrame([result["results"] for result in custom_results.values()])
results_df[cols_to_show]

In [ ]:
results_df

In [ ]:
import plotly.express as px

# Create a new column with custom hover text
results_df['custom_hover_text'] = results_df.apply(lambda row: f"""
Pair: {row['config']['trading_pair']}
<br>Volume: {row['total_volume']}
<br>PNL: {row['net_pnl_quote']}
<br>Close types: {row['close_types']}

Config:
<br>Id: {row['config']['id']}
<br>BB Length: {row['config']['bb_length']}
<br>BB Std: {row['config']['bb_std']}
<br>Take profit: {row['config']['take_profit']}
<br>Stop loss: {row['config']['stop_loss']}
<br>Time limit: {row['config']['time_limit']}
<br>DCA Spreads: {row['config']['dca_spreads']}
<br>DCA Amounts: {row['config']['dca_amounts_pct']}
""", axis=1)

# Create the scatter plot with the custom hover text
fig = px.scatter(
    results_df,
    x="total_volume",
    y="net_pnl_quote",
    color="trading_pair",  # Color by trading_pair
    hover_data={"custom_hover_text": True},  # Show the custom hover text
    color_discrete_sequence=px.colors.qualitative.Plotly,  # Optional: use a specific color sequence
    title="Net PNL Quote vs. Total Volume by Trading Pair"
)

# Show the figure
fig.show()


In [ ]:
results_df.sort_values("net_pnl_quote", ascending=False).iloc[0]

In [ ]:
config_to_backtest = results_df.sort_values("net_pnl_quote", ascending=False).iloc[0]["config"]
bt_results_r = bt_results[config_to_backtest["id"] +]

In [ ]:
bt_results

In [ ]:
bt_results.keys()

In [ ]:
import pickle

path = os.path.join(root_path, "research_notebooks", "xtreet_bb", "backtesting_results_2.pickle")
with open("backtesting_results_2.pickle", "wb") as f:
    pickle.dump(bt_results, f)

In [ ]:
bt_results_r = bt_results["shupoalgo.yml"]
df_r = bt_results_r["df"]
config_r = bt_results_r["config"]
executors_r = bt_results_r["executors"]

In [ ]:
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from hummingbot.connector.connector_base import TradeType


def get_bt_candlestick_trace(df):
    df.index = pd.to_datetime(df.timestamp, unit='s')
    return go.Scatter(x=df.index,
                      y=df['close'],
                      mode='lines',
                      line=dict(color="blue"),
                      )


def get_pnl_trace(executors):
    pnl = [e.net_pnl_quote for e in executors]
    cum_pnl = np.cumsum(pnl)
    return go.Scatter(
        x=pd.to_datetime([e.close_timestamp for e in executors], unit="s"),
        y=cum_pnl,
        mode='lines',
        line=dict(color='gold', width=2, dash="dash"),
        name='Cumulative PNL'
    )


def get_default_layout(title=None, height=800, width=1800):
    layout = {
        "template": "plotly_dark",
        "plot_bgcolor": 'rgba(0, 0, 0, 0)',  # Transparent background
        "paper_bgcolor": 'rgba(0, 0, 0, 0.1)',  # Lighter shade for the paper
        "font": {"color": 'white', "size": 12},  # Consistent font color and size
        "height": height,
        "width": width,
        "margin": {"l": 20, "r": 20, "t": 50, "b": 20},
        "xaxis_rangeslider_visible": False,
        "hovermode": "x unified",
        "showlegend": False,
    }
    if title:
        layout["title"] = title
    return layout


def add_executors_trace(fig, executors, row, col):
    for executor in executors:
        entry_time = pd.to_datetime(executor.timestamp, unit='s')
        entry_price = executor.custom_info["current_position_average_price"]
        exit_time = pd.to_datetime(executor.close_timestamp, unit='s')
        filled_time = pd.to_datetime(executor.custom_info["entry_timestamp"], unit='s')
        exit_price = executor.custom_info["close_price"]
        name = "Buy Executor" if executor.config.side == TradeType.BUY else "Sell Executor"
        if executor.config.side == TradeType.BUY:
            stop_loss = entry_price * (1 - float(executor.config.stop_loss))
            take_profit = entry_price * (1 + float(executor.config.take_profit))
        else:
            stop_loss = entry_price * (1 + float(executor.config.stop_loss))
            take_profit = entry_price * (1 - float(executor.config.take_profit))

        if executor.filled_amount_quote == 0:
            fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[entry_price, entry_price], mode='lines',
                                     line=dict(color='grey', width=2, dash="dash"), name=name), row=row, col=col)
        else:
            if executor.net_pnl_quote > Decimal(0):
                fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[entry_price, exit_price], mode='lines',
                                         line=dict(color='green', width=3), name=name), row=row, col=col)
            else:
                fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[entry_price, exit_price], mode='lines',
                                         line=dict(color='red', width=3), name=name), row=row, col=col)
            fig.add_trace(go.Scatter(x=[filled_time, filled_time], y=[entry_price, exit_price], mode='lines',
                                     line=dict(color='lightcoral', width=2), name=name), row=row, col=col)
            fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[stop_loss, stop_loss], mode='lines',
                                     line=dict(color='red', width=1, dash="dash"), name=name), row=row, col=col)
            fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[take_profit, take_profit], mode='lines',
                                     line=dict(color='green', width=1, dash="dash"), name=name), row=row, col=col)
    return fig


def create_backtesting_figure(df, executors, config):
    # Create subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.02, subplot_titles=('Candlestick', 'PNL Quote'),
                        row_heights=[0.7, 0.3])

    # Add candlestick trace
    fig.add_trace(get_bt_candlestick_trace(df), row=1, col=1)

    # Add executors trace
    fig = add_executors_trace(fig, executors, row=1, col=1)

    # Add PNL trace
    fig.add_trace(get_pnl_trace(executors), row=2, col=1)

    # Apply the theme layout
    layout_settings = get_default_layout(f"Trading Pair: {config['trading_pair']}")
    layout_settings["showlegend"] = False
    fig.update_layout(**layout_settings)

    # Update axis properties
    fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
    fig.update_xaxes(row=2, col=1)
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="PNL", row=2, col=1)
    return fig


fig = create_backtesting_figure(
    df=df_r,
    executors=executors_r,
    config=config_r.dict())
# df.ta.bbands(length=config.bb_length, std=config.bb_std, append=True)
fig.add_trace(go.Scatter(x=df_r.index,
                         y=df_r[f"BBU_{config_r.bb_length}_{config_r.bb_std}"])
              )
fig.add_trace(go.Scatter(x=df_r.index,
                         y=df_r[f"BBM_{config_r.bb_length}_{config_r.bb_std}"])
              )
fig.add_trace(go.Scatter(x=df_r.index,
                         y=df_r[f"BBL_{config_r.bb_length}_{config_r.bb_std}"])
              )
fig.update_layout(width=1400)
fig.show()